In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import tensorflow as tf


print(f'\nTensorflow version = {tf.__version__}\n')
print(f'\n{tf.config.list_physical_devices("GPU")}\n')


# Загрузка данных
data = pd.read_csv('data/my_train.csv')

# Разделение на train и valid
train_texts, valid_texts, train_labels, valid_labels = train_test_split(
    data['text'], data['label'], test_size=0.2, random_state=42
)

# Создание датасета
train_dataset = Dataset.from_dict({'text': train_texts.tolist(), 'label': train_labels.tolist()})
valid_dataset = Dataset.from_dict({'text': valid_texts.tolist(), 'label': valid_labels.tolist()})



Tensorflow version = 2.17.1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]



In [6]:
# Инициализация токенизатора
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

# Токенизация
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize_function, batched=True)
valid_dataset = valid_dataset.map(tokenize_function, batched=True)

# Удаление неиспользуемых колонок
train_dataset = train_dataset.remove_columns(["text"])
valid_dataset = valid_dataset.remove_columns(["text"])
train_dataset.set_format("torch")
valid_dataset.set_format("torch")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/4727 [00:00<?, ? examples/s]

Map:   0%|          | 0/1182 [00:00<?, ? examples/s]

#Обучаем классификатор

In [8]:
# Инициализация модели
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=2)

# Определение метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    cm = confusion_matrix(labels, predictions)
    return {"accuracy": acc, "f1": f1, "confusion_matrix": cm.tolist()}

# Настройки обучения
training_args = TrainingArguments(
    output_dir="./classifier",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-f7be808c8b07>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Confusion Matrix
1,0.305200,0.232385,0.866328,0.863183,"[[613, 0], [158, 411]]"
2,0.220200,0.242259,0.884941,0.883737,"[[595, 18], [118, 451]]"
3,0.145800,0.226646,0.879865,0.879898,"[[526, 87], [55, 514]]"
4,0.124100,0.226050,0.892555,0.892590,"[[536, 77], [50, 519]]"
5,0.383900,0.199108,0.889171,0.889186,"[[545, 68], [63, 506]]"
6,0.185900,0.191824,0.901015,0.900963,"[[561, 52], [65, 504]]"
7,0.121300,0.265664,0.899323,0.899297,"[[557, 56], [63, 506]]"
8,0.107000,0.230882,0.900169,0.900186,"[[551, 62], [56, 513]]"


TrainOutput(global_step=4728, training_loss=0.2043408863813744, metrics={'train_runtime': 1020.6808, 'train_samples_per_second': 37.05, 'train_steps_per_second': 4.632, 'total_flos': 2504693573787648.0, 'train_loss': 0.2043408863813744, 'epoch': 8.0})

In [10]:
# Сохраняем модель
model.save_pretrained("./classifier")

#Претренируем модель на train-test.txt

In [11]:
from transformers import DataCollatorForLanguageModeling

# Загрузка текста
with open('data/train-test.txt', 'r') as file:
    texts = file.readlines()

text_data = list(map(lambda x: x.split('. '), texts))[0]

dataset = Dataset.from_dict({"text": text_data})

def filter_short_texts(example):
    return len(example['text'].split()) >=3

dataset = dataset.filter(filter_short_texts)

dataset = dataset.train_test_split(test_size=0.3)


Filter:   0%|          | 0/23397 [00:00<?, ? examples/s]

In [12]:
# Токенизация данных
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/15859 [00:00<?, ? examples/s]

Map:   0%|          | 0/6797 [00:00<?, ? examples/s]

In [13]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.

    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

lm_dataset = tokenized_dataset.map(group_texts, batched=True)

Map:   0%|          | 0/15859 [00:00<?, ? examples/s]

Map:   0%|          | 0/6797 [00:00<?, ? examples/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
from transformers import AutoModelForMaskedLM

# Инициализация модели
mlm_model = AutoModelForMaskedLM.from_pretrained("distilbert/distilroberta-base")

# Настройки обучения
mlm_training_args = TrainingArguments(
    output_dir="./retrained",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=7,
    logging_steps=10,
    report_to="none"
)

mlm_trainer = Trainer(
    model=mlm_model,
    args=mlm_training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)


mlm_trainer.train()

# Метрики perplexity и loss
eval_results = mlm_trainer.evaluate()



config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-15-2efbb5a14775>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `

Epoch,Training Loss,Validation Loss
1,0.177700,0.211121
2,0.212100,0.181862


In [ ]:
import math
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 3.33


In [4]:
# Сохраняем перетренированную модель
mlm_model.save_pretrained("./retrained")

NameError: name 'model' is not defined

#Перетренировываем классификатор на весах второй модели

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

# Перетренированная модель с MLM
pretrained_model = AutoModelForSequenceClassification.from_pretrained(
    "./retrained",
    num_labels=2
    )

for param in mlm_model.base_model.parameters():
    param.requires_grad = False

for param in mlm_model.roberta.encoder.layer[-3:].parameters():
    param.requires_grad = True

pretrained_training_args = TrainingArguments(
    output_dir="./final_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=7,
    logging_steps=10,
    report_to="none"
)

# Новый тренер
trainer = Trainer(
    model=pretrained_model,
    args=pretrained_training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

# Сохранение модели
trainer.save_model("final_model")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./second and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-23-450873c00f05>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.